In [ ]:
# データの永続化
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
%cd '/content/drive/My Drive/work/'

In [ ]:
# TensorFlow 1.xへの切り替え
%tensorflow_version 1.x

In [ ]:
# image-gptのクローン
!git clone https://github.com/openai/image-gpt.git

In [ ]:
%%time

# 事前学習済みモデルとカラーパレットのダウンロード
!python image-gpt/download.py --model s --ckpt 1000000 --cluster --download_dir download

In [ ]:
from PIL import Image
from imageio import imwrite
import numpy as np

# カラーパレットの読み込み
clusters = np.load("download/kmeans_centers.npy")

# RGB画像の読み込み
im = Image.open('image.png')
im = im.resize((32,32)) # リサイズ
imr = np.array(im) # NumPy配列への変換

# 2点間の距離の計算
def dist(x, y):
    return np.sqrt(np.sum((x-y)**2))

# RGB色をカラーパレットのインデックスに変換
def find_index(a):
    mind = 10000  # 最小距離
    minidx = -1   # 最小距離のインデックス
    for i in range(len(clusters)): 
        d = dist(a/127.5-1.0, clusters[i])
        if mind > d:
            mind = d
            minidx = i
    return minidx

# RGB画像をパレット画像に変換
result = []
for y in range(32):
    for x in range(32):
        result.append(find_index(imr[y, x]))

# パレット画像の保存
samples = np.array(result) # NumPy配列への変換
np.save("palette_image.npy",samples) # 保存

# 確認用にパレット画像をRGB画像に変換して表示
samples = np.reshape(np.rint(127.5*(clusters[samples]+1.0)), [32, 32, 3]).astype(np.uint8)
imwrite('check_image.png', samples)
Image.open('check_image.png')

In [ ]:
%%time

# 画像の生成
!python image-gpt/src/run.py --sample --n_embd=512 --n_head=8 --n_layer=24 --ckpt_path=download/model.ckpt-1000000 --color_cluster_path=download/kmeans_centers.npy --n_gpu=1 --n_sub_batch=8

In [ ]:
# 結果画像の表示
Image.open('result.png')